<a href="https://colab.research.google.com/github/mohammadbadi/Clustering_FE_MCA/blob/main/Code%20Sections/5.0%20Loading%20Libraries%20and%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### **5.0 Loading Libraries and Major Crime Indicator Dataset from TPS**

In [ ]:
                                                                                  # Import necessary libraries
import itertools
import os
import pandas as pd
import time
import kagglehub
import warnings
from IPython.display import display, HTML
from kagglehub import KaggleDatasetAdapter
from google.colab import files

os.system('pip install openpyxl -qqq')                                            # Install openpyxl for Excel support
os.system('pip install tabulate -qqq')                                            # Install tabulate for cleaner table output

warnings.filterwarnings("ignore", category=DeprecationWarning)                    # Ignore Deprecation Warnings
warnings.filterwarnings("ignore", category=FutureWarning)                         # Ignore future warnings

file_path = "major-crime-indicators.csv"                                          # Set the file path to the filename with extension

crime_df = kagglehub.load_dataset(                                                # Load the latest version of the dataset from Kaggle
    kagglehub.KaggleDatasetAdapter.PANDAS,
    "mohammadbadi/crimes-in-toronto",                                             # Updated dataset handle
    file_path,
)

def format_message(message):                                                      # Function to format HTML messages
    return f"""
<div style="font-size: 18px; color: #333; font-weight: bold; padding: 10px;">
    {message}
</div>
"""

load_message = format_message(                                                    # Display HTML formatted message confirming that the dataset is loaded
    "Dataset <span style='color: blue;'>major-crime-indicators.csv</span> by <span style='color: slategray;'>Mohammad Badi</span> from Kaggle website is <span style='color: green;'>Successfully</span> loaded!"
)
display(HTML(load_message))

crime_df.to_csv("major-crime-indicators.csv", index=False)                        # Save the loaded dataset as a CSV file

save_message = format_message(                                                    # Display HTML formatted message confirming that the dataset is saved
    "Dataset saved in <span style='color: blue;'>current workspace</span> <span style='color: green;'>Successfully!</span>"
)
display(HTML(save_message))

major_crime_df = crime_df                                                         # Reusing the dataframe loaded earlier
def save_data(data_df, filename_base):
    csv_filename = f"{filename_base}.csv"                                         # Save as CSV
    data_df.to_csv(csv_filename, index=False)
    csv_msg = format_message(f"Data saved as CSV: <span style='color: blue;'>{csv_filename}</span>")
    display(HTML(csv_msg))
    excel_filename = f"{filename_base}.xlsx"                                      # Save as Excel
    data_df.to_excel(excel_filename, index=False, engine='openpyxl')
    excel_msg = format_message(f"Data saved as Excel: <span style='color: blue;'>{excel_filename}</span>")
    display(HTML(excel_msg))

save_data(major_crime_df, "Checking_Load_Time")                                   # Save the dataset as both CSV and Excel with name 'Checking_Load_Time'

def measure_read_time(file_path, file_type):                                      # Function to measure file reading time
    start_time = time.time()
    if file_type == "csv":
        pd.read_csv(file_path)
    elif file_type == "excel":
        pd.read_excel(file_path)
    end_time = time.time()
    return end_time - start_time

csv_time = measure_read_time('Checking_Load_Time.csv', "csv")                     # Measure read times
excel_time = measure_read_time('Checking_Load_Time.xlsx', "excel")

csv_time_color = "green" if csv_time < excel_time else "red"                      # Determine color coding for time messages
excel_time_color = "green" if excel_time < csv_time else "red"

csv_time_message = format_message(                                                # Display HTML formatted time messages
    f"Time taken to read <span style='color: blue;'>Checking_Load_Time CSV file</span>: <span style='color: {csv_time_color};'>{csv_time:.2f} seconds</span>"
)

excel_time_message = format_message(
    f"Time taken to read <span style='color: blue;'>Checking_Load_Time Excel file</span>: <span style='color: {excel_time_color};'>{excel_time:.2f} seconds</span>"
)

display(HTML(csv_time_message))                                                   # Display the time messages
display(HTML(excel_time_message))

if csv_time < excel_time:                                                         # Determine the recommendation based on time
    speed_factor = excel_time / csv_time
    recommendation = (
        f"Recommendation: Load the data from <span style='color: green;'>CSV</span> as it is approximately "
        f"<span style='color: green;'>{speed_factor:.2f} times faster</span> than loading from Excel."
    )
else:
    speed_factor = csv_time / excel_time
    recommendation = (
        f"Recommendation: Load the data from <span style='color: green;'>Excel</span> as it is approximately "
        f"<span style='color: green;'>{speed_factor:.2f} times faster</span> than loading from CSV."
    )

recommendation_message = format_message(recommendation)
display(HTML(recommendation_message))

completion_message = format_message("Dataset has been analyzed, and recommendation has been provided!")
display(HTML(completion_message))


### **5.1 Learning Application Domain: Summary Table**

In [1]:
import warnings                                                                   # Import necessary libraries
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import os
import contextlib
from google.colab import files
from IPython.display import display, HTML
from kagglehub import KaggleDatasetAdapter

print("\n\n")
# Suppress warnings
warnings.simplefilter(action='ignore', category=FutureWarning)                    # Ignore Deprecation Warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)                    # Ignore future warnings

file_path = "major-crime-indicators.csv"                                          # File path for the dataset

with open(os.devnull, 'w') as fnull:                                              # Load the latest version using the correct dataset handle while suppressing the download output
    with contextlib.redirect_stdout(fnull):
        df = kagglehub.load_dataset(
            KaggleDatasetAdapter.PANDAS,
            "mohammadbadi/crimes-in-toronto",                                     # Dataset handle on Kaggle
            file_path,
        )

def count_leading_trailing_spaces(column):                                        # Function to check leading/trailing spaces, ensuring the column is treated as a string
    column = column.astype(str)
    return column.str.startswith(' ').sum(), column.str.endswith(' ').sum()

def prepare_summary_table(df):                                                    # Function to prepare the summary table with only Null values check and calculate % of Null
    unique_values = df.nunique()                                                  # Count total, unique, and Null values for each column
    total_values = df.count() + df.isnull().sum()                                 # Total includes NaN and Null
    null_counts = df.isnull().sum()

    null_percentages = (null_counts / total_values) * 100                         # Calculate the percentage of Null values compared to total values

    leading_spaces, trailing_spaces = zip(*[count_leading_trailing_spaces(df[col]) for col in df.columns]) # Initialize column-based space checks

    summary_table = pd.DataFrame({                                                # Create a summary table with the calculated values
        "Column": df.columns,
        "Data Type": df.dtypes,
        "Total Values": total_values,
        "Unique Values": unique_values,
        "Null Values": null_counts,
        "Null %": null_percentages.round(1),
        "Leading Spaces": leading_spaces,
        "Trailing Spaces": trailing_spaces
    })

    return summary_table, null_counts, null_percentages

def generate_summary_html(summary_table):                                        # Function to generate HTML for the summary table
    summary_table_html = """
    <style>
        table {
            border-collapse: collapse;
            width: 100%;
            font-family: Arial, sans-serif;
        }
        table th, table td {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }
        table th {
            background-color: #4CAF50;
            color: white;
            font-size: 1.1em;
        }
        table tr:nth-child(even) {background-color: #f2f2f2;}
        table tr:hover {background-color: #ddd;}
    </style>
    <table>
        <tr>
            <th>Column</th>
            <th>Data Type</th>
            <th>Total Values</th>
            <th>Unique Values</th>
            <th>Null Values</th>
            <th>Null %</th>
            <th>Leading Spaces</th>
            <th>Trailing Spaces</th>
        </tr>
    """

    for _, row in summary_table.iterrows():
        null_color = "red" if row['Null Values'] > 0 else "green"
        leading_color = "red" if row['Leading Spaces'] > 0 else "green"
        trailing_color = "red" if row['Trailing Spaces'] > 0 else "green"

        summary_table_html += f"""
        <tr>
            <td>{row['Column']}</td>
            <td>{row['Data Type']}</td>
            <td>{row['Total Values']}</td>
            <td>{row['Unique Values']}</td>
            <td style='color:{null_color};'>{row['Null Values']}</td>
            <td>{row['Null %']}</td>
            <td style='color:{leading_color};'>{row['Leading Spaces']}</td>
            <td style='color:{trailing_color};'>{row['Trailing Spaces']}</td>
        </tr>
        """
    summary_table_html += "</table>"
    return summary_table_html

def plot_missing_percentage(df, dataset_name):                                    # Function to plot missing percentage visualization
    missing_percentage = df.isnull().mean() * 100                                 # Calculate the missing percentage for each column
    missing_percentage = missing_percentage[missing_percentage > 0]               # show only columns with missing values

    if missing_percentage.empty:                                                  # Check if there's any column with missing data
        print(f"No missing data in {dataset_name}. Skipping missing percentage plot.")
        return ""

    plt.figure(figsize=(10, 6))                                                  # Plotting the bar chart for missing percentages
    missing_percentage.sort_values().plot(kind='barh', color='skyblue', edgecolor='grey')
    plt.title(f'Missing Data Percentage - {dataset_name}', fontsize=16)
    plt.xlabel('Missing Percentage (%)', fontsize=12)
    plt.ylabel('Columns', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    missing_plot_path = f'/content/5.1 Original Data Missing PercentPlot.png'     # Path to save the plot
    plt.savefig(missing_plot_path)
    plt.close()                                                                   # Close the plot to avoid displaying it again

    return missing_plot_path

def get_suggestions(null_counts):                                                 # Function to generate suggestions based on Null values
    suggestions = []

    no_null_columns = null_counts[null_counts == 0].index.tolist()                # Handle columns with no Null values
    if no_null_columns:
        suggestions.append(f"<div style='color:green;'><strong>Columns {', '.join(no_null_columns)}</strong> have no Null values. <em>No action is needed.</em></div>")

    columns_with_null_data = null_counts[null_counts > 0].index.tolist()          # Handle columns with Null values
    if columns_with_null_data:
        suggestions.append(f"<div style='color:red;'><strong>Columns {', '.join(columns_with_null_data)}</strong> have Null values. <em>Actions should be taken.</em></div>")

    return suggestions

def display_summary_for_file(df, title):                                          # Function to display the summary table and suggestions
    summary_table, null_counts, null_percentages = prepare_summary_table(df)      # Prepare the summary table
    summary_html = generate_summary_html(summary_table)

    suggestions = get_suggestions(null_counts)                                    # Get the suggestions based on Null values handling

    missing_plot_path = plot_missing_percentage(df, title)                        # Plot the missing percentage visual and get the path of the image
                                                                                  # Combine only the table and suggestions into the HTML content
    complete_html = f"""
    <html>
    <head><title>Dataset Summary - {title}</title></head>
    <body>
        <h2>Summary Table for {title}</h2>
        {summary_html}
        <h2>Suggestions</h2>
        {"".join(suggestions)}
    </body>
    </html>
    """
    summary_html_path = f'/content/5.1 Learning Application Domain.html'          # Save the HTML output to a file for table + suggestions
    with open(summary_html_path, 'w', encoding='utf-8') as f:
        f.write(complete_html)

    return summary_html_path, missing_plot_path

html_file_path, missing_plot_path = display_summary_for_file(df, "Analysis on Original Data") # Save and display summary for dataset

display(HTML(f"<h2>Summary Table and Suggestions for Major Crime Indicators Dataset</h2><br>{open(html_file_path).read()}"))  # Display the summary table and suggestions in the console

files.download(html_file_path)                                                    # Download the generated Summary Table + Suggestions as HTML
files.download(missing_plot_path)                                                 # Download the generated Missing Data Percentage plot as PNG
print("\n\n")

100%|██████████| 117M/117M [00:08<00:00, 15.1MB/s]


Column,Data Type,Total Values,Unique Values,Null Values,Null %,Leading Spaces,Trailing Spaces
_id,int64,420200,420200,0,0.0,0,0
EVENT_UNIQUE_ID,object,420200,366257,0,0.0,0,0
REPORT_DATE,object,420200,4018,0,0.0,0,0
OCC_DATE,object,420200,4518,0,0.0,0,0
REPORT_YEAR,int64,420200,11,0,0.0,0,0
REPORT_MONTH,object,420200,12,0,0.0,0,0
REPORT_DAY,int64,420200,31,0,0.0,0,0
REPORT_DOY,int64,420200,366,0,0.0,0,0
REPORT_DOW,object,420200,7,0,0.0,0,420200
REPORT_HOUR,int64,420200,24,0,0.0,0,0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **5.2	Creating Target Dataset**

In [2]:
import warnings                                                                   # Import necessary libraries
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import os
import contextlib
from google.colab import files
from IPython.display import display, HTML
from kagglehub import KaggleDatasetAdapter

print("\n\n")
warnings.filterwarnings("ignore", category=DeprecationWarning)                    # Ignore Deprecation Warnings
warnings.filterwarnings("ignore", category=FutureWarning)                         # Ignore future warnings

file_path = "major-crime-indicators.csv"                                          # Set the file path to the filename with extension

crime_df = kagglehub.load_dataset(                                                # Load the latest version of the dataset from Kaggle
    kagglehub.KaggleDatasetAdapter.PANDAS,
    "mohammadbadi/crimes-in-toronto",                                             # Updated dataset handle
    file_path,
)

initial_count = df.shape[0]

filter1_df = df[(df['UCR_CODE'] == 2135) & (df['UCR_EXT'] == 210)].copy()         # Filter 1: UCR Code 2135 with UCR Extension 210
count1 = filter1_df.shape[0]

filter2_df = df[(df['UCR_CODE'] == 1610) & (df['UCR_EXT'] == 140)].copy()         # Filter 2: UCR Code 1610 with UCR Extension 140
count2 = filter2_df.shape[0]

final_df = pd.concat([filter1_df, filter2_df]).copy()                             # Final dataset: Union of both filters
final_count = final_df.shape[0]

final_df.to_csv('Target_Dataset.csv', index=False)                                # Save the final dataset as Target_Dataset.csv
files.download('Target_Dataset.csv')                                              # Download the saved dataset csv

steps_summary = []                                                                # Build steps summary as a list of dictionaries
steps_summary.append({
    "Step Taken": "Filter 1: UCR Code 2135 with UCR Extension 210",
    "Before Action": initial_count,
    "Affected by Action": count1,
    "After Action": count1,
    "Unit": "Rows"
})
steps_summary.append({
    "Step Taken": "Filter 2: UCR Code 1610 with UCR Extension 140",
    "Before Action": count1,                                                      # using count1 as the 'After Action' of Filter 1
    "Affected by Action": count2,
    "After Action": final_count,
    "Unit": "Rows"
})
steps_summary.append({
    "Step Taken": "Rows Affected in <strong>UCR Filtering</strong>",
    "Before Action": "Initial Load:<br><strong>" + str(initial_count) + "</strong>",
    "Affected by Action": "Rows Filtered:<br><strong>" + str(initial_count - final_count) + "</strong>",
    "After Action": "Final Count:<br><strong>" + str(final_count) + "</strong>",
    "Unit": "Rows"
})

html_output_filename = '/content/5.2 Target Dataset.html'                         # Set the HTML output filename
                                                                                  # Create HTML Table with styling
html_table = """
<table style='border-collapse: collapse; width: 100%; font-size: 18px;'>
    <thead style='background-color: #4CAF50; color: white;'>
        <tr>
            <th colspan="5" style="text-align: center; font-size: 24px; background-color: #2f4f4f; color: white;">
                5.2 Creating Target Dataset
            </th>
        </tr>
        <tr>
            <th>Step Taken</th>
            <th>Before Action</th>
            <th>Affected by Action</th>
            <th>After Action</th>
            <th>Unit</th>
        </tr>
    </thead>
    <tbody>
"""

for step in steps_summary:
    html_table += f"""
    <tr style='border: 1px solid #dddddd;'>
        <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Step Taken']}</td>
        <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Before Action']}</td>
        <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Affected by Action']}</td>
        <td style='border: 1px solid #dddddd; padding: 8px;'>{step['After Action']}</td>
        <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Unit']}</td>
    </tr>
    """

note_text = (                                                                     # Add a final row with the note (spanning all columns)
    "<strong>Note: The dataset contains "
    "<span style='color: darkred; '>ALL CRIMES</span>, but our research focuses on "
    "<span style='color: green; '>MOTOR VEHICLE THEFTS</span>. "
    "Therefore, we applied two filters: <br>"
    "• Filter 1: UCR Code 2135 with UCR Extension 210 for Theft of a Motor Vehicle (Auto Theft), and <br>"
    "• Filter 2: UCR Code 1610 with UCR Extension 140 for Robbery - Vehicle Jacking.<br>"
    "The target dataset has been saved as <span style='color: blue;'>'Target_Dataset.csv'</span> for further analysis. </strong>"
)
html_table += f"""
    <tr style='border: 1px solid #dddddd;'>
        <td colspan="5" style='border: 1px solid #dddddd; padding: 8px;'>{note_text}</td>
    </tr>
"""
html_table += "</tbody></table>"


print("\n\n")
display(HTML(html_table))                                                          # Display the output HTML table

with open(html_output_filename, 'w', encoding='utf-8') as f:
    f.write(html_table)
files.download(html_output_filename)
print("\n\n")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **5.3 Data Cleaning**

In [3]:
import warnings                                                                   # Import necessary libraries
import pandas as pd
from IPython.display import display, HTML
from google.colab import files

warnings.filterwarnings("ignore", category=DeprecationWarning)                    # Ignore Deprecation Warnings
warnings.filterwarnings("ignore", category=FutureWarning)                         # Ignore future warnings

url = "https://raw.githubusercontent.com/mohammadbadi/CrimeAnalytics_Clustering/refs/heads/main/Output_CSV/Target_Dataset.csv"    # Read the data from CSV file
Data_Preparing_df = pd.read_csv(url, low_memory=False).copy()

print("\n\n")

html_output_filename = '/content/5.3 Data Cleaning.html'                          # Output File Name for HTML summary changed to "5.3 Data Cleaning"

steps_summary = []                                                                # Table to store results

before_step_1 = Data_Preparing_df.shape[0]                                        # Step 1: Dataset Loading
steps_summary.append({
    "Step Taken": "Step 1: Load Dataset",
    "Before Action": before_step_1,
    "Affected by Action": 0,
    "After Action": before_step_1,
    "Unit": "Rows"
})

columns_to_check = [col for col in Data_Preparing_df.columns if col != '_id']     # Step 2: Identify and remove true duplicates (excluding '_id')
duplicate_count = Data_Preparing_df.duplicated(subset=columns_to_check).sum()
rows_before_dedup = Data_Preparing_df.shape[0]
Data_Preparing_df = Data_Preparing_df.drop_duplicates(subset=columns_to_check, keep='first').copy()
rows_after_dedup = Data_Preparing_df.shape[0]
steps_summary.append({
    "Step Taken": "Step 2: Remove TRUE DUPLICATE Records",
    "Before Action": rows_before_dedup,
    "Affected by Action": duplicate_count,
    "After Action": rows_after_dedup,
    "Unit": "Rows"
})

before_drop_rows = Data_Preparing_df.shape[0]                                     # Step 3: Drop rows with null, NaN, or missing data
Data_Preparing_df = Data_Preparing_df.dropna().copy()
after_drop_rows = Data_Preparing_df.shape[0]
steps_summary.append({
    "Step Taken": "Step 3: Drop Rows with Missing Data",
    "Before Action": before_drop_rows,
    "Affected by Action": before_drop_rows - after_drop_rows,
    "After Action": after_drop_rows,
    "Unit": "Rows"
})

obj_cols = Data_Preparing_df.select_dtypes(include="object").columns              # Step 4: Strip leading and trailing spaces from string columns
orig_step4 = Data_Preparing_df[obj_cols].copy()
Data_Preparing_df[obj_cols] = Data_Preparing_df[obj_cols].apply(lambda s: s.str.strip())
affected_step4 = (orig_step4 != Data_Preparing_df[obj_cols]).any(axis=1).sum()
steps_summary.append({
    "Step Taken": "Step 4: Strip Leading/Trailing Spaces",
    "Before Action": after_drop_rows,
    "Affected by Action": affected_step4,
    "After Action": after_drop_rows,
    "Unit": "Rows"
})

orig_step5 = Data_Preparing_df[obj_cols].copy()                                   # Step 5: Remove leading apostrophes from string columns
Data_Preparing_df[obj_cols] = Data_Preparing_df[obj_cols].apply(lambda s: s.str.lstrip("'"))
affected_step5 = (orig_step5 != Data_Preparing_df[obj_cols]).any(axis=1).sum()
steps_summary.append({
    "Step Taken": "Step 5: Remove Leading Apostrophes",
    "Before Action": after_drop_rows,
    "Affected by Action": affected_step5,
    "After Action": after_drop_rows,
    "Unit": "Rows"
})

nsa_replaced_count = 0                                                            # Step 6: Match rows where 'HOOD_158' is 'NSA' and replace with matching value based on coordinates
for i, row in Data_Preparing_df[Data_Preparing_df['HOOD_158'] == 'NSA'].iterrows():
    match = Data_Preparing_df[
        (Data_Preparing_df['LONG_WGS84'] == row['LONG_WGS84']) &
        (Data_Preparing_df['LAT_WGS84'] == row['LAT_WGS84']) &
        (Data_Preparing_df['HOOD_158'] != 'NSA')
    ]
    if not match.empty:
        matched_value = match.iloc[0]['HOOD_158']
        Data_Preparing_df.loc[i, 'HOOD_158'] = matched_value
        nsa_replaced_count += 1
steps_summary.append({
    "Step Taken": "Step 6: Match & Replace 'NSA' Values",
    "Before Action": after_drop_rows,
    "Affected by Action": nsa_replaced_count,
    "After Action": after_drop_rows,
    "Unit": "Rows"
})

mask_remaining_nsa = (Data_Preparing_df['HOOD_158'] == 'NSA') | (Data_Preparing_df['NEIGHBOURHOOD_158'] == 'NSA')     # Step 7: Remove remaining rows where 'HOOD_158' or 'NEIGHBOURHOOD_158' contains 'NSA'
remaining_nsa_count = mask_remaining_nsa.sum()
before_removal = Data_Preparing_df.shape[0]
Data_Preparing_df = Data_Preparing_df[~mask_remaining_nsa].copy()
after_removal = Data_Preparing_df.shape[0]
steps_summary.append({
    "Step Taken": "Step 7: Remove Remaining 'NSA' Rows",
    "Before Action": before_removal,
    "Affected by Action": remaining_nsa_count,
    "After Action": after_removal,
    "Unit": "Rows"
})

orig_long = Data_Preparing_df['LONG_WGS84'].copy()                                # Step 8: Format Longitude & Latitude to 7 decimals
orig_lat = Data_Preparing_df['LAT_WGS84'].copy()
Data_Preparing_df.loc[:, 'LONG_WGS84'] = Data_Preparing_df['LONG_WGS84'].astype(float).map(lambda x: f"{x:.7f}")
Data_Preparing_df.loc[:, 'LAT_WGS84'] = Data_Preparing_df['LAT_WGS84'].astype(float).map(lambda x: f"{x:.7f}")
affected_step8 = ((orig_long != Data_Preparing_df['LONG_WGS84']) | (orig_lat != Data_Preparing_df['LAT_WGS84'])).sum()
steps_summary.append({
    "Step Taken": "Step 8: Format Longitude & Latitude to 7 Decimals",
    "Before Action": after_removal,
    "Affected by Action": affected_step8,
    "After Action": after_removal,
    "Unit": "Rows"
})

final_row_count = Data_Preparing_df.shape[0]                                      # Final row: Rows Affected in 5.3 Data Cleaning
steps_summary.append({
    "Step Taken": "Rows Affected in <strong>5.3 Data Cleaning </strong>",
    "Before Action": "Initial Load:<br><strong>" + str(before_step_1) + "</strong>",
    "Affected by Action": "Overall Reduction:<br><strong>" + str(before_step_1 - final_row_count) + "</strong>",
    "After Action": "Final Count:<br><strong>" + str(final_row_count) + "</strong>",
    "Unit": "Rows"
})
                                                                                  # Build HTML Table with styling
html_table = """
<table style='border-collapse: collapse; width: 100%; font-size: 18px;'>
    <thead style='background-color: #4CAF50; color: white;'>
        <tr>
            <th colspan="5" style="text-align: center; font-size: 24px; background-color: #2f4f4f; color: white;">
                5.3 Data Cleaning
            </th>
        </tr>
        <tr>
            <th>Step Taken</th>
            <th>Before Action</th>
            <th>Affected by Action</th>
            <th>After Action</th>
            <th>Unit</th>
        </tr>
    </thead>
    <tbody>
"""
for step in steps_summary:
    html_table += f"""
        <tr style='border: 1px solid #dddddd;'>
            <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Step Taken']}</td>
            <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Before Action']}</td>
            <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Affected by Action']}</td>
            <td style='border: 1px solid #dddddd; padding: 8px;'>{step['After Action']}</td>
            <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Unit']}</td>
        </tr>
    """
note_text = (                                                                     # Add final note row spanning all columns with the required note text
    "<strong>Note: "
    "Longitude and Latitude were reduced to 7 Decimal Places as,<br>"
    "• 7 decimal places offer precision of <span style='color: green;'>1.1 cm</span>, precise enough for GPS devices.<br>"
    "• Further granularity adds processing time and energy consumption without real-world benefits.<br>"
    "The final cleaned data has been saved as <span style='color: blue;'> 'Cleaned_Data.csv' </span> for further analysis.</strong>"
)
html_table += f"""
        <tr style='border: 1px solid #dddddd;'>
            <td colspan="5" style='border: 1px solid #dddddd; padding: 8px;'>{note_text}</td>
        </tr>
    </tbody>
</table>
"""

Data_Preparing_df.to_csv('Cleaned_Data.csv', index=False)                         # Save the cleaned data to a new CSV file

display(HTML(html_table))                                                         # Display the HTML table

with open(html_output_filename, 'w', encoding='utf-8') as f:                      # Save the HTML table to a file
    f.write(html_table)
files.download(html_output_filename)                                              # Download the HTML file
files.download('Cleaned_Data.csv')                                                # Download the cleaned CSV file
print("\n\n")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>